<a href="https://colab.research.google.com/github/penny1xu/RESTS/blob/main/python_rests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# data clean 

BathroomsFull
BathroomsHalf
BathroomsTotalDecimal
BathroomsTotalInteger
BedroomsTotal
BuildingAreaTotal
City
CloseDate
ClosePrice
CumulativeDaysOnMarket
DaysOnMarket
GarageSpaces
Latitude
ListingContractDate
ListingId
ListPrice
ListSource
Longitude
LotSizeAcres
MlsStatus
OwnerName
OwnerPhone
OwnerPhoneAlternative
ParcelNumber
ParcelNumber2
PoolYN
PostalCode
PreviousListPrice
PreviousStatus
PropertySubType
PropertyType
PublicRemarks
PurchaseContractDate
SchoolDistrict
StandardStatus
StatusChangeTimestamp
StreetDirPrefix
StreetDirSuffix
StreetName
StreetNumber
StreetNumberNumeric
StreetSuffix
SubdivisionName
USProperty_MUI
YearBuilt



In [ ]:
#import package
!pip install censusgeocode
import csv
import pandas as pd
import censusgeocode as cg
import numpy as np


# Data cleaning

In [ ]:
data= pd.read_csv("final.csv")
# select the data columns we want 
data = data[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt','LivingArea']]


In [6]:
# change data type
zip = []
lo = []
la = []

for item in data['PostalCode']:
    item = str(item)
    zip.append(item)
for item in data['Longitude']:
    item = float(item)
    lo.append(item)
for item in data['Latitude']:
    item = float(item)
    la.append(item)
data['PostalCode'] = zip
data['Longitude'] = lo
data['Latitude'] = la

In [7]:
# choose columns status

data = data.loc[data['PropertySubType'] == 'Single Family Residence' ]
data = data.loc[data['PropertyType'] == 'Residential']

# Set Input

In [51]:
# input data address
print('Enter #MLS')
mls = 14730621
print('Enter address')
address = '2836 Meadowbrook Drive'
print('Enter city')
city = 'Dallas'
print('Enter state')
state = 'Texas'
print('Enter zipcode')
zip = '75227'



Enter #MLS
Enter address
Enter city
Enter state
Enter zipcode


# filter the data

In [53]:
# find value in csv 
res_mls = data.isin([mls]).any().any()
if res_mls :
  print("\nThis value exists in Dataframe")
  # get whole row value
  pulled_value = data.loc[data['ListingId'] == mls]
  #get each value
  dict = pulled_value.to_dict('records')

  # # limit zipcode range
  data = data.loc[data['PostalCode'] == zip ]
 

# get each line value

#call the value from the dict
  


In [54]:
else :
  print("\nThis value does not exists in Dataframe")
  # # limit zipcode range
  data = data.loc[data['PostalCode'] == zip ]
  # if there is no longitude 
  #get Longitude and Latitude

  ##cg = CensusGeocode(benchmark='Public_AR_Current', vintage='Current_Current')
  result = cg.address(address, city = city, state= state,zipcode = zip)
  lo = result[0]['coordinates']['x']
  la = result[0]['coordinates']['y']
  print(result[0]['coordinates'])
  # limit longitude and latitude
  if 
  new_data = data[(data['Longitude'] > lo - 0.01) & (data['Longitude'] < lo + 0.01)&
               (data['Latitude'] < la + 0.02)& (data['Latitude'] > la - 0.02)]


SyntaxError: ignored

In [ ]:
# limit the days

data = data[data['CumulativeDaysOnMarket'].isin(range(0,90))] 

In [ ]:
# limit square feet

if 'LivingArea' <= 1100:
  data[(data['LivingArea'] > area - 100) & (data['LivingArea'] < area + 100)
elif 'LivingArea' <= 2200:
  data[(data['LivingArea'] > area -200 )& (data['LivingArea'] > area + 200)]
else:
  data[(data['LivingArea'] > area -300 )& (data['LivingArea'] > area + 300)]


In [43]:
#defined number
house_leftLa = dict[0].get('Latitude')
house_rightLa = dict[0].get('Latitude') 
house_leftLo = dict[0].get('Longitude')
house_rightLo = dict[0].get('Longitude')
house_leftSqt = dict[0].get('LivingArea')
house_rightSqt = dict[0].get('LivingArea')

# code part

In [50]:
from numpy.ma.core import append
def sortByLongAndLat (houses,rightSqt,rightLa,leftLa,leftLo,rightLo):
  result = pd.houses()
  for house in houses :
    if house_leftLo >= leftLo && house_rightLo <= rightLo && house_leftLa >= leftLa && house_rightLa <= rightLa:
      result.append(house)
  return result




SyntaxError: ignored

In [41]:

def findHouse ():
   count = int(0)
   leftLa = house_leftLa
   rightLa = house_rightLa 
   leftLo = house_leftLo
   rightLo = house_rightLo
   leftSqt = house_leftSqt
   rightSqt = house_rightSqt
   zip = dict[0].get('PostalCode')
   # create an Empty DataFrame object
   result = pd.houses()
   selectedHouse = data.loc[data['PostalCode'] == zip ]


# if the total value return less than 5
   while count <= 5 {
       
       
   }

    




